read_sample lee una muestra a la vez

Espera hasta timeout para devolver un error si no logra leer la muestra con el mensaje:
SampleAcquisitionTimeout: **Time out occurred while waiting for a start byte**

Da la impresión de que no hay buffer, si leo el puerto serie a tiempos superiores al intervalo entre muestras se pierden muestras. OpenBCI lo dice a través del byte de parada que no será 0x0C y tenemos acceso porque en la estructura de datos que envía por cada muestra el campo Valid = False

**IMPORTANTE** El ID de muestra es de 8 bits, se reinicia a 0 cada 256 muestras
** Secuencia**
cyton.read_sample -> cyton.read_packet -> cyton.wait_start_byte -> pyserial.Serial.read(1)

pyserial.Serial.read(size) se duerme hasta que entra valor por puerto serie o ocurre timeout y devuelve la cantidad de bytes que haya logrado leer

dentro de cyton.wait_start_byte se lanza exception SampleAcquisitionTimeout cuando el pyserial.Serial.read() ha llegado al timeout y por lo tanto no devuelve ningún valor

Estrategias:
1. Probar con solamente una muestra
2. 




In [1]:
import time
import pandas as pd
from openbci_interface import Cyton
from openbci_interface.util import list_devices
from serial import Serial
import pickle

# from saccrec.consts import DEBUG

"""
def list_ports():
    if DEBUG:
        return [
            '/dev/ttyUSB0',
            '/dev/ttyUSB1',
        ]
    return [port for port in list_devices()]
"""
def init_board():
    sample_rate = 250
    puerto = '/dev/ttyUSB0'
    port = Serial(
        port=puerto,
        baudrate=115200,
        timeout=2
    )

    print("configurando placa...\n")
    lista = []
    cnt = 0
    with Cyton(port) as board:
        board.set_board_mode('default')
        board.set_sample_rate(sample_rate)
        board.disable_channel(5)
        board.disable_channel(6)
        board.disable_channel(7)
        board.disable_channel(8)
        board.configure_channel(1, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
        board.configure_channel(2, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
        board.configure_channel(3, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
        board.configure_channel(4, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
        print(board.get_config())        
        board.start_streaming()
        for x in range(500):
            sample = board.read_sample()
            cnt += 1
            lista.append(sample)
            # time.sleep(0.85 / sample_rate)
        board.stop_streaming()
        fichero = open('datos.pickle', 'wb') 
        pickle.dump(lista,fichero)
        fichero.close()
        return lista, cnt

In [3]:
data, cnt = init_board()

configurando placa...

{'board_mode': 'default', 'sample_rate': 250, 'channels': [{'enabled': True, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 0, 'srb2': 0, 'srb1': 1}}, {'enabled': True, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 0, 'srb2': 0, 'srb1': 1}}, {'enabled': True, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 0, 'srb2': 0, 'srb1': 1}}, {'enabled': True, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 0, 'srb2': 0, 'srb1': 1}}, {'enabled': False, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 1, 'srb2': 1, 'srb1': 0}}, {'enabled': False, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 1, 'srb2': 1, 'srb1': 0}}, {'enabled': False, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 1, 'srb2': 1, 'srb1': 0}}, {'enabled': False, 'parameters': {'power_down':

In [4]:
cnt

500

In [82]:
data[-1]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [12]:
p = []
from openbci_interface import util
p = util.list_devices()
p

<generator object list_devices at 0x7f310f5efa20>